<a href="https://colab.research.google.com/github/tsjendoll/predict_health_costs_with_regression/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
X = pd.get_dummies(dataset)
y = X.pop('expenses')

In [ ]:
X

In [ ]:
y = y.astype(int)
y

In [ ]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X,y, test_size=0.20, random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_error

class MyRandomForestClassifier:
  def __init__(self, n_estimators, max_depth, random_state):
    self.model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)

  def evaluate(self, X,y, verbose=2):
    y_pred = self.predict(X)
    mae = mean_absolute_error(y, y_pred)
    return mae

  def fit(self, X, y):
    self.model.fit(X, y)

  def predict(self, X):
    return self.model.predict(X)

def get_mae(n_estimators, max_depth, random_state, train_X, val_X, train_y, val_y):
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=random_state)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [ ]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
final_X_train = pd.DataFrame(my_imputer.fit_transform(train_X))
final_X_valid = pd.DataFrame(my_imputer.transform(val_X))

In [ ]:
train_labels = train_y
test_labels = val_y
train_dataset = train_X
test_dataset = val_X

In [ ]:
n_estimators_list = [150,200,250,300]
max_depth_list = [2,3,4,5,6,7,8,9]
random_state_list = [2,3,4,5,6,7]
mae_list = []
for n_estimators in n_estimators_list:
    for max_depth in max_depth_list:
        for random_state in random_state_list:
            mae = get_mae(n_estimators, max_depth, random_state, final_X_train, final_X_valid, train_y, val_y)
            #print(f'No. Estimators: {n_estimators} \t Max Depth: {max_depth} \t Mean Absolute Error: {mae}')
            mae_list.append([n_estimators, max_depth, random_state, mae])
best_result = min(mae_list, key=lambda x:x[3])
print(f"No. Estimators: {best_result[0]} \n Max Depth: {best_result[1]} \n Random State: {best_result[2]} Mean Absolute Error: {best_result[3]}")

In [ ]:
model = MyRandomForestClassifier(n_estimators=best_result[0], max_depth=best_result[1], random_state=best_result[2])
model.fit(final_X_train,train_y)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
mae = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
